In [1]:
# move workidir to the correct folder
import os
os.chdir("../")

import numpy as np
import pandas as pd
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

# Custom Libs
from dataset.dataloader import Dataloader

In [2]:
# Set up which conlumn to use as a target :
# 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
# 'review_scores_communication', 'review_scores_location', 'review_scores_value',

# Note: not defined for scores

REFERENCE_COlUMN = "review_scores_rating"
BUKETS = [(0, 4), (4, 4.5), (4.5, 5)]

In [3]:
# Loading the dataset
DATASET_PATH = f'dataset'

data = pd.read_csv('dataset/listings_with_comments.csv')
# Taking only text data 
X = data[['name', 'description', 'host_about' , 'neighborhood_overview', 'comments']]
# Extracting the choosen conlumn
Y = data[REFERENCE_COlUMN].apply(lambda f: float(f))

# removing null references
nan_idx = Y.notna()
X = X[nan_idx]
Y = Y[nan_idx]

# Fill empty conlum with empty text
X.fillna("")


,name,description,host_about,neighborhood_overview,comments
0,Quiet Garden View Room & Super Fast Wi-Fi,Quiet Garden View Room & Super Fast Wi-Fi<br /...,"Upon arriving in Amsterdam, one can imagine as...","Indische Buurt (""Indies Neighborhood"") is a ne...",['Daniel is really cool. The place was nice an...
1,*historic centre* *bright* *canal view* *jordaan*,> Please be so kind to book ONLY AFTER conta...,"We love cycling, travelling, healthy food and ...",Perfect location in the lively centre. All his...,['Nice apartment in a beautifull área of Amste...
2,Elegant Appartement Central Location.,"From December 26 thru January 2, minimum stay ...",I like to welcome guests who take good care of...,Bij aankomst vertel ik mijn gasten graag over ...,"['Einfach große Klasse! Zentrale Lage, Einkauf..."
3,beautiful designed ap.+bikes+garden,Cosy and comfortable apartment on the ground f...,"We are the parents of 4 beautiful children, tw...",'De Pijp' as we Amsterdammers know it today is...,"[""This was by far the best Airbnb experience I..."
4,Lovely FAMILY house near Vondelpark,* Our house is only suitable for a couple or f...,"Welcome guests, I'm Sebastiaan and it would be...",,"['Amazing house, perfect location and a dream ..."
...,...,...,...,...,...
6895,Studio in centre of Amsterdam,"Studio in the centre of Amsterdam, everything ...",,,['Ótima localização']
6900,Cozy and comfortable appartment,Het centraal gelegen appartment is in 1930 geb...,,,"['חי', 'apparement très agréable et pratique d..."
6910,Spacious apartment near De Pijp,Spacious apartment perfect for a couple or sol...,,,['L’appartement d’Angus est très cosy et fonct...
6929,Urban getaway Amsterdam,Deze centraal gelegen accommodatie is smaakvol...,I'm Gunni and I will be your host during your ...,,"[""Really nice apartment. Good location. Very c..."


In [4]:
x_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42) # Note all the train test split are taken with same seed

In [5]:
def bucketizie(x):
    for i in range(len(BUKETS)):
        start, end = BUKETS[i]
        if (i == 0): 
            if x >= start and  x<=end: return i
        else:
            if x > start and  x<=end: return i

y_train_buckets = [bucketizie(val) for val in y_train]
y_test_buckets = [bucketizie(val) for val in y_test]

np.unique(y_train_buckets)

array([0, 1, 2])

## Comments Vs Description
We train 2 sentence models 1 for the commnets and one for the description usings sets of sentencies for each possible bin taking around 100 of them each.
So the sentence model is trained using BatchAllTripletLoss tat will try to train the language model Bi-Encoding all the elements in all the possible classes.

In [6]:
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

model_comments = SentenceTransformer('distiluse-base-multilingual-cased-v2')
model_descriptions = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Generation of the datasets for classification by samplig the data by the bins ranges
train_examples_comments = []
train_examples_descriptions = []
for i, b in enumerate(BUKETS):
     start, end = b
     in_range = x_train[y_train > start][y_train <= end] # selection of groups by target values bins

     train_examples_comments += [InputExample(texts= in_range.description.sample(100).values, label=i)]
     train_examples_descriptions += [InputExample(texts=in_range.comments.sample(100).values, label=i)]

train_dataloader_comments = DataLoader(train_examples_comments, shuffle=True, batch_size=16)
train_dataloader_description = DataLoader(train_examples_descriptions, shuffle=True, batch_size=16)

train_loss_comments = losses.BatchAllTripletLoss(model_comments)
train_loss_descriptions = losses.BatchAllTripletLoss(model_descriptions)

model_comments.fit(train_objectives=[(train_dataloader_comments, train_loss_comments)], epochs=10) 
model_descriptions.fit(train_objectives=[(train_dataloader_description, train_loss_descriptions)], epochs=10) 

/tmp/ipykernel_94306/3048723032.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  in_range = x_train[y_train > start][y_train <= end] # selection of groups by target values bins
/tmp/ipykernel_94306/3048723032.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  in_range = x_train[y_train > start][y_train <= end] # selection of groups by target values bins
/tmp/ipykernel_94306/3048723032.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  in_range = x_train[y_train > start][y_train <= end] # selection of groups by target values bins


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

## Classification using k-Means

In [10]:
from sklearn.cluster import KMeans
# On Comments
corpus = X_test.comments.values
corpus_embeddings_comments = model_comments.encode(corpus)
# Perform kmean clustering
num_clusters = len(BUKETS)
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings_comments)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(y_test[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

1264 1264


/home/mangelini/miniconda3/envs/NLP/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyError: 0

In [ ]:
from sklearn.cluster import KMeans
# On Descriptions
corpus = X_test.description.values
corpus_embeddings_comments = model_comments.encode(corpus)

# Perform kmean clustering
num_clusters = len(BUKETS)
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings_comments)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

/home/mangelini/miniconda3/envs/NLP/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster  1
['Lovely modern two room (1 bedroom) apartment on the second floor overlooking the hip Javaplein. The apartment is 50m2. Within walking distance (< 1 minute) to one of the popular shopping boulevard’s in Oost, the Javastraat, where nice restaurants and shops are located. The real Amsterdam feeling! Luxurious bathroom and kitchen, you will feel instantly at home. The tram/bus stop is right downstairs (<10 minutes to Rembrandsquare and Leidsesquare).<br /><br /><b>The space</b><br />Please note: the second bed is a convertible couch to bed in the living room.<br /><br /><b>License number</b><br />0363 77D1 0FFE FCD4 3D1E', 'Let op: alleen voor kattenliefhebbers! <br />Een ruime gezinswoning met tuin en twee balkons in hartje Amsterdam. Tussen het Vondelpark en het Rembrandtpark. In een rustige buurt maar met restaurants, winkeltjes en het uitgaancentrum om de hoek.<br />Ons huis heeft een ruime woonkamer, een open leefkeuken, drie lichte slaapkamers, een werkkamer en een prach